In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
df_warehouse=pd.read_excel(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\INPUT_DATA_FILE\WAREHOUSE_DATA_FILE.xlsx")

In [3]:
df_warehouse.to_csv("warehouse_data.csv",index=False)

In [4]:
warehouse_Data = pd.read_csv("C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\warehouse_data.csv")

In [5]:
assest_initiation_invent_sb = pd.read_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\INPUT_DATA_FILE\ASSET_INITIATION_INVENTORY_SB.CSV")

In [6]:
assest_initiation_invent_nb = pd.read_csv(r"C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\INPUT_DATA_FILE\ASSET_INITIATION_INVENTORY_NB.CSV")

In [7]:
assest_initiation_invent_sb.count()

MeterNumber                                  995983
MFG_SERIAL_NUM                               995983
UDC_ID                                       995711
NETWORK_ID                                   995711
InstallationNumber                           962014
Status                                       995983
INSERT_TIME                                  995983
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    995983
dtype: int64

# change data type into string

In [8]:
# Convert all columns to string data type

warehouse_Data = warehouse_Data.astype(str)
assest_initiation_invent_nb = assest_initiation_invent_nb.astype(str)
assest_initiation_invent_sb = assest_initiation_invent_sb.astype(str)


# change column name into upper string

In [9]:
warehouse_Data.columns = warehouse_Data.columns.str.upper()
assest_initiation_invent_nb.columns = assest_initiation_invent_nb.columns.str.upper()
assest_initiation_invent_sb.columns = assest_initiation_invent_sb.columns.str.upper()

# trim the trailling and leading spaces of needed column

In [10]:
# Remove leading and trailing whitespaces from all columns where dtype is 'O'
warehouse_Data = warehouse_Data.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
assest_initiation_invent_nb = assest_initiation_invent_nb.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
assest_initiation_invent_sb = assest_initiation_invent_sb.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)


# strip the spaces in column 

In [11]:




# strip the spaces in column 

warehouse_Data.columns = warehouse_Data.columns.str.strip()
assest_initiation_invent_nb.columns = assest_initiation_invent_nb.columns.str.strip()
assest_initiation_invent_sb.columns = assest_initiation_invent_sb.columns.str.strip()



In [12]:
warehouse_Data.count()

METER NO.             4000
BOX NUMBER            4000
FINAL SIM             4000
IME                   4000
PHASE                 4000
IMSI                  4000
IP                    4000
WAREHOUSE_UTILITY     4000
MAKE                  4000
LOT BY LOT            4000
IMPORTATION STATUS    4000
dtype: int64

In [13]:
warehouse_Data.columns

Index(['METER NO.', 'BOX NUMBER', 'FINAL SIM', 'IME', 'PHASE', 'IMSI', 'IP',
       'WAREHOUSE_UTILITY', 'MAKE', 'LOT BY LOT', 'IMPORTATION STATUS'],
      dtype='object')

In [ ]:
# concat the assest_initiation_invent_sb and assest_initiation_invent_nb

NB_SB_COMBINED_DATAFRAME = pd.concat([assest_initiation_invent_sb, assest_initiation_invent_nb], ignore_index=True)

In [ ]:
# checking is null

NB_SB_COMBINED_DATAFRAME[NB_SB_COMBINED_DATAFRAME["NETWORK_ID"].isnull()].count()

METERNUMBER                                  0
MFG_SERIAL_NUM                               0
UDC_ID                                       0
NETWORK_ID                                   0
INSTALLATIONNUMBER                           0
STATUS                                       0
INSERT_TIME                                  0
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    0
dtype: int64

In [ ]:
# checking duplicate value

NB_SB_COMBINED_DATAFRAME.duplicated().sum()

0

In [ ]:
# droping not used column in the NB_SB_COMBINED_DATAFRAME

NB_SB_COMBINED_DATAFRAME.drop(['METERNUMBER', 'INSTALLATIONNUMBER','INSERT_TIME'], axis=1, inplace=True)

In [ ]:
NB_SB_COMBINED_DATAFRAME.head()

,MFG_SERIAL_NUM,UDC_ID,NETWORK_ID,STATUS,"DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"
0,GP4970294,404759210464358,2001:4490:0800:5F25:0000:0000:0000:0001,Meter Shop,SBPDCL
1,GP4962320,404751250451741,2001:4490:0800:A6F5:0000:0000:0000:1,Installed,SBPDCL
2,GP4964119,404751250456722,2001:4490:0800:B5B4:0000:0000:0000:0001,Installed,SBPDCL
3,GP4964992,404751250456883,2001:4490:0800:B655:0000:0000:0000:0001,Installed,SBPDCL
4,GP4964995,404751250456886,2001:4490:0800:B658:0000:0000:0000:0001,Installed,SBPDCL


In [ ]:
NB_SB_COMBINED_DATAFRAME.count()

MFG_SERIAL_NUM                               1762518
UDC_ID                                       1762518
NETWORK_ID                                   1762518
STATUS                                       1762518
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    1762518
dtype: int64

In [ ]:
# dop duplicate in  NB_SB_COMBINED_DATAFRAME

NB_SB_COMBINED_DATAFRAME.drop_duplicates(subset=['MFG_SERIAL_NUM','UDC_ID',"DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"], keep='first', inplace=True, ignore_index=True)

In [ ]:
NB_SB_COMBINED_DATAFRAME.count()[0]

C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_22944\1323936542.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  NB_SB_COMBINED_DATAFRAME.count()[0]


1699438

In [ ]:


# Convert all columns to string data type

NB_SB_COMBINED_DATAFRAME = NB_SB_COMBINED_DATAFRAME.astype(str)


NB_SB_COMBINED_DATAFRAME.columns = NB_SB_COMBINED_DATAFRAME.columns.str.upper()



# Remove leading and trailing whitespaces from all columns where dtype is 'O'
NB_SB_COMBINED_DATAFRAME = NB_SB_COMBINED_DATAFRAME.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)


In [15]:
#assest_initiation_invent_sb[assest_initiation_invent_sb['UDC_ID']=='405856706290441']

# JOIN



In [ ]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA = pd.merge(warehouse_Data, NB_SB_COMBINED_DATAFRAME, left_on=['METER NO.', 'WAREHOUSE_UTILITY'], right_on=['MFG_SERIAL_NUM', "DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"], how='left')


In [ ]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.count()

METER NO.                                    4000
BOX NUMBER                                   4000
FINAL SIM                                    4000
IME                                          4000
PHASE                                        4000
IMSI                                         4000
IP                                           4000
FINAL HES MAPPING                            4000
MAKE                                         4000
LOT BY LOT                                   4000
IMPORTATION STATUS                           4000
MFG_SERIAL_NUM                               1271
UDC_ID                                       1271
NETWORK_ID                                   1271
STATUS                                       1271
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    1271
dtype: int64

In [ ]:
#LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.to_csv(r'C:\Users\Ratan Kumar Jha\Desktop\new\LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.csv',index=False)

In [ ]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA = pd.merge(warehouse_Data, NB_SB_COMBINED_DATAFRAME, left_on=['IMSI'], right_on=['UDC_ID'], how='left')

In [ ]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.count()

METER NO.                                    4000
BOX NUMBER                                   4000
FINAL SIM                                    4000
IME                                          4000
PHASE                                        4000
IMSI                                         4000
IP                                           4000
FINAL HES MAPPING                            4000
MAKE                                         4000
LOT BY LOT                                   4000
IMPORTATION STATUS                           4000
MFG_SERIAL_NUM                               3996
UDC_ID                                       3996
NETWORK_ID                                   3996
STATUS                                       3996
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    3996
dtype: int64

# duplicate row while we join with imsi 

In [ ]:
#LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[~LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA['UDC_ID'].isnull()].count()

In [ ]:
# duplicated row ..  bcz in asset initiation file there are 2 installtion no.  with respect to 1 imsi no.

duplicate_check = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA['UDC_ID'].duplicated(keep=False)]

duplicate_check.to_csv(r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\duplicate_check_wrt_IMSI_IN_ASSET_FIL.csv',index=False)
duplicate_check.to_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\duplicate_check_wrt_IMSI_IN_ASSET_FIL.csv",index=False)

In [ ]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.count()


METER NO.                                    4000
BOX NUMBER                                   4000
FINAL SIM                                    4000
IME                                          4000
PHASE                                        4000
IMSI                                         4000
IP                                           4000
FINAL HES MAPPING                            4000
MAKE                                         4000
LOT BY LOT                                   4000
IMPORTATION STATUS                           4000
MFG_SERIAL_NUM                               3996
UDC_ID                                       3996
NETWORK_ID                                   3996
STATUS                                       3996
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    3996
dtype: int64

In [ ]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.drop_duplicates(subset=['IMSI'],inplace=True)


In [ ]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.count()

METER NO.                                    4000
BOX NUMBER                                   4000
FINAL SIM                                    4000
IME                                          4000
PHASE                                        4000
IMSI                                         4000
IP                                           4000
FINAL HES MAPPING                            4000
MAKE                                         4000
LOT BY LOT                                   4000
IMPORTATION STATUS                           4000
MFG_SERIAL_NUM                               3996
UDC_ID                                       3996
NETWORK_ID                                   3996
STATUS                                       3996
DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')    3996
dtype: int64

In [ ]:
#LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA['METER NO.']=='LT0783643']

##### joining both the table "   LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA   "       and            " LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA "

In [ ]:
# LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA : column rename

In [ ]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.rename(
    columns={"UDC_ID": "METER_MATCH_UDC_ID", "NETWORK_ID": "METER_MATCH_IP", "STATUS": "METER_MATCH_STATUS","DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')":"METER_MATCH_UTILITY"},
    inplace=True
)



In [ ]:
# LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA : column in modified column

In [ ]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMINED_COLUMN=LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA.columns
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMINED_COLUMN

Index(['METER NO.', 'BOX NUMBER', 'FINAL SIM', 'IME', 'PHASE', 'IMSI', 'IP',
       'FINAL HES MAPPING', 'MAKE', 'LOT BY LOT', 'IMPORTATION STATUS',
       'MFG_SERIAL_NUM', 'METER_MATCH_UDC_ID', 'METER_MATCH_IP',
       'METER_MATCH_STATUS', 'METER_MATCH_UTILITY'],
      dtype='object')

In [ ]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMINED_COLUMN]

In [ ]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head()

,METER NO.,BOX NUMBER,FINAL SIM,IME,PHASE,IMSI,IP,FINAL HES MAPPING,MAKE,LOT BY LOT,IMPORTATION STATUS,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY
0,GP4971073,BOX NO- 1805P,8991759065212008001,869912032848513,1 Phase,404751250008001,fd00:0000:0800:DBB9:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,GP4971073,404751250453398,2001:4490:0800:ACA4:0000:0000:0000:0001,Retired,NBPDCL
1,GP5015712,BOX NO- 1805P,8991759065212008002,869912033630167,1 Phase,404751250008002,fd00:0000:0800:DBBA:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,NaN,NaN,NaN,NaN,NaN
2,GP5015622,BOX NO- 1805P,8991759065212008003,869912033743713,1 Phase,404751250008003,fd00:0000:0800:DBBB:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,NaN,NaN,NaN,NaN,NaN
3,GP5019777,BOX NO- 1805P,8991759065212008004,869912033458346,1 Phase,404751250008004,fd00:0000:0800:DBBC:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,NaN,NaN,NaN,NaN,NaN
4,GP5011165,BOX NO- 1805P,8991759065212008005,869912033521200,1 Phase,404751250008005,fd00:0000:0800:DBBD:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,NaN,NaN,NaN,NaN,NaN


In [ ]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED[LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED['METER_MATCH_UDC_ID']=='nan'].count()

METER NO.              0
BOX NUMBER             0
FINAL SIM              0
IME                    0
PHASE                  0
IMSI                   0
IP                     0
FINAL HES MAPPING      0
MAKE                   0
LOT BY LOT             0
IMPORTATION STATUS     0
MFG_SERIAL_NUM         0
METER_MATCH_UDC_ID     0
METER_MATCH_IP         0
METER_MATCH_STATUS     0
METER_MATCH_UTILITY    0
dtype: int64

In [ ]:
#LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA[['SL. NO.','METER NO.','PHASE TYPE','FINAL HES MAPPING','CATEGORY FOR DASHBOARD','SUPPLIER','IME AS PER L&T SHIPMENT','SIM 19 DIST','IMSI','IP','REMARKS',"METER_MATCH_UDC_ID","METER_MATCH_IP","METER_MATCH_STATUS","METER_MATCH_UTILITY"]]

In [ ]:
# LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA : column rename

In [ ]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.rename(
    columns = {"MFG_SERIAL_NUM":"IMSI_MATCHED_METER_NO","NETWORK_ID":"IMSI_MATCHED_IP","STATUS":"IMSI_MATCHED_STATUS","DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')":"IMSI_MATCHED_UTILITY"},
    inplace=True
)

In [ ]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.head(1)

,METER NO.,BOX NUMBER,FINAL SIM,IME,PHASE,IMSI,IP,FINAL HES MAPPING,MAKE,LOT BY LOT,IMPORTATION STATUS,IMSI_MATCHED_METER_NO,UDC_ID,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY
0,GP4971073,BOX NO- 1805P,8991759065212008001,869912032848513,1 Phase,404751250008001,fd00:0000:0800:DBB9:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,GP4971073,404751250008001,fd00:0000:0800:DBB9:0000:0000:0000:0001,Meter Shop,SBPDCL


In [ ]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_COLUMN=LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA.columns
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_COLUMN

Index(['METER NO.', 'BOX NUMBER', 'FINAL SIM', 'IME', 'PHASE', 'IMSI', 'IP',
       'FINAL HES MAPPING', 'MAKE', 'LOT BY LOT', 'IMPORTATION STATUS',
       'IMSI_MATCHED_METER_NO', 'UDC_ID', 'IMSI_MATCHED_IP',
       'IMSI_MATCHED_STATUS', 'IMSI_MATCHED_UTILITY'],
      dtype='object')

In [ ]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_COLUMN]

In [ ]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(1)

,METER NO.,BOX NUMBER,FINAL SIM,IME,PHASE,IMSI,IP,FINAL HES MAPPING,MAKE,LOT BY LOT,IMPORTATION STATUS,IMSI_MATCHED_METER_NO,UDC_ID,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY
0,GP4971073,BOX NO- 1805P,8991759065212008001,869912032848513,1 Phase,404751250008001,fd00:0000:0800:DBB9:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,GP4971073,404751250008001,fd00:0000:0800:DBB9:0000:0000:0000:0001,Meter Shop,SBPDCL


In [ ]:
#LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA[['SL. NO.','METER NO.','PHASE TYPE','FINAL HES MAPPING','CATEGORY FOR DASHBOARD','SUPPLIER','IME AS PER L&T SHIPMENT','SIM 19 DIST','IMSI','IP','REMARKS',"IMSI_MATCHED_METER_NO","IMSI_MATCHED_IP","IMSI_MATCHED_STATUS","IMSI_MATCHED_UTILITY"]]

### WORK ON LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED  and  LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED

In [ ]:
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(3)

,METER NO.,BOX NUMBER,FINAL SIM,IME,PHASE,IMSI,IP,FINAL HES MAPPING,MAKE,LOT BY LOT,IMPORTATION STATUS,IMSI_MATCHED_METER_NO,UDC_ID,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY
0,GP4971073,BOX NO- 1805P,8991759065212008001,869912032848513,1 Phase,404751250008001,fd00:0000:0800:DBB9:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,GP4971073,404751250008001,fd00:0000:0800:DBB9:0000:0000:0000:0001,Meter Shop,SBPDCL
1,GP5015712,BOX NO- 1805P,8991759065212008002,869912033630167,1 Phase,404751250008002,fd00:0000:0800:DBBA:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,GP5015712,404751250008002,fd00:0000:0800:DBBA:0000:0000:0000:0001,Refurbished,SBPDCL
2,GP5015622,BOX NO- 1805P,8991759065212008003,869912033743713,1 Phase,404751250008003,fd00:0000:0800:DBBB:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,GP5015622,404751250008003,fd00:0000:0800:DBBB:0000:0000:0000:0001,Refurbished,SBPDCL


In [ ]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(3)

,METER NO.,BOX NUMBER,FINAL SIM,IME,PHASE,IMSI,IP,FINAL HES MAPPING,MAKE,LOT BY LOT,IMPORTATION STATUS,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY
0,GP4971073,BOX NO- 1805P,8991759065212008001,869912032848513,1 Phase,404751250008001,fd00:0000:0800:DBB9:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,GP4971073,404751250453398,2001:4490:0800:ACA4:0000:0000:0000:0001,Retired,NBPDCL
1,GP5015712,BOX NO- 1805P,8991759065212008002,869912033630167,1 Phase,404751250008002,fd00:0000:0800:DBBA:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,NaN,NaN,NaN,NaN,NaN
2,GP5015622,BOX NO- 1805P,8991759065212008003,869912033743713,1 Phase,404751250008003,fd00:0000:0800:DBBB:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,NaN,NaN,NaN,NaN,NaN


In [ ]:
#

# data correction

In [ ]:
# Convert all columns to string data type

# change into str
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.astype(str)
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.astype(str)


# ----------------------------------------------------------------------------------

# change into upper
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns.str.upper()
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.columns.str.upper()


# ----------------------------------------------------------------------------------

# Remove leading and trailing whitespaces from all columns where dtype is 'O'
LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED = LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)




In [ ]:
LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED.head(1)

,METER NO.,BOX NUMBER,FINAL SIM,IME,PHASE,IMSI,IP,FINAL HES MAPPING,MAKE,LOT BY LOT,IMPORTATION STATUS,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY
0,GP4971073,BOX NO- 1805P,8991759065212008001,869912032848513,1 Phase,404751250008001,fd00:0000:0800:DBB9:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,GP4971073,404751250453398,2001:4490:0800:ACA4:0000:0000:0000:0001,Retired,NBPDCL


# Merge IMSI  and METER  file

In [ ]:
warehouse_meter_imsi_data= pd.DataFrame(columns=[warehouse_Data.columns])

In [ ]:
warehouse_meter_imsi_data

,METER NO.,BOX NUMBER,FINAL SIM,IME,PHASE,IMSI,IP,FINAL HES MAPPING,MAKE,LOT BY LOT,IMPORTATION STATUS


In [ ]:
#  empty column add to the  warehouse_meter_imsi_data  which has already some empty column of warehouse data

In [ ]:
warehouse_meter_imsi_data['METER_MATCH_UDC_ID'] = None
warehouse_meter_imsi_data['METER_MATCH_IP'] = None
warehouse_meter_imsi_data['METER_MATCH_STATUS'] = None
warehouse_meter_imsi_data['METER_MATCH_UTILITY'] = None
warehouse_meter_imsi_data['IMSI_MATCHED_METER_NO'] = None
warehouse_meter_imsi_data['IMSI_MATCHED_IP'] = None
warehouse_meter_imsi_data['IMSI_MATCHED_STATUS'] = None
warehouse_meter_imsi_data['IMSI_MATCHED_UTILITY'] = None


In [ ]:
warehouse_meter_imsi_data

,METER NO.,BOX NUMBER,FINAL SIM,IME,PHASE,IMSI,IP,FINAL HES MAPPING,MAKE,LOT BY LOT,IMPORTATION STATUS,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY


In [ ]:
# this column are not same in all the dataframe so that i used the above method

In [ ]:
"""
warehouse_meter_imsi_data = pd.DataFrame(columns=['SL. NO.', 'METER NO.', 'PHASE TYPE', 'WAREHOUSE_UTILITY',
       'CATEGORY FOR DASHBOARD', 'SUPPLIER', 'IME AS PER L&T SHIPMENT',
       'SIM 19 DIST', 'IMSI', 'IP', 'REMARKS', 'METER_MATCH_UDC_ID',
       'METER_MATCH_IP', 'METER_MATCH_STATUS', 'METER_MATCH_UTILITY',
       'IMSI_MATCHED_METER_NO', 'IMSI_MATCHED_IP', 'IMSI_MATCHED_STATUS',
       'IMSI_MATCHED_UTILITY'])
"""

"\nwarehouse_meter_imsi_data = pd.DataFrame(columns=['SL. NO.', 'METER NO.', 'PHASE TYPE', 'FINAL HES MAPPING',\n       'CATEGORY FOR DASHBOARD', 'SUPPLIER', 'IME AS PER L&T SHIPMENT',\n       'SIM 19 DIST', 'IMSI', 'IP', 'REMARKS', 'METER_MATCH_UDC_ID',\n       'METER_MATCH_IP', 'METER_MATCH_STATUS', 'METER_MATCH_UTILITY',\n       'IMSI_MATCHED_METER_NO', 'IMSI_MATCHED_IP', 'IMSI_MATCHED_STATUS',\n       'IMSI_MATCHED_UTILITY'])\n"

In [ ]:
warehouse_meter_imsi_data.head()

,METER NO.,BOX NUMBER,FINAL SIM,IME,PHASE,IMSI,IP,FINAL HES MAPPING,MAKE,LOT BY LOT,IMPORTATION STATUS,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY


In [ ]:
warehouse_meter_imsi_data = pd.merge(LEFT_JOIN_WITH_METER_UTILITY_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED, LEFT_JOIN_WITH_IMSI_WAREHOUSE_AND_COMIBNED_DATA_MODIFIED[['METER NO.', 'IMSI','IMSI_MATCHED_METER_NO','IMSI_MATCHED_IP','IMSI_MATCHED_STATUS','IMSI_MATCHED_UTILITY']], left_on=['METER NO.', 'IMSI'], right_on=['METER NO.', 'IMSI'], how='left')

In [ ]:
warehouse_meter_imsi_data.head(2)

,METER NO.,BOX NUMBER,FINAL SIM,IME,PHASE,IMSI,IP,FINAL HES MAPPING,MAKE,LOT BY LOT,IMPORTATION STATUS,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY
0,GP4971073,BOX NO- 1805P,8991759065212008001,869912032848513,1 Phase,404751250008001,fd00:0000:0800:DBB9:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,GP4971073,404751250453398,2001:4490:0800:ACA4:0000:0000:0000:0001,Retired,NBPDCL,GP4971073,fd00:0000:0800:DBB9:0000:0000:0000:0001,Meter Shop,SBPDCL
1,GP5015712,BOX NO- 1805P,8991759065212008002,869912033630167,1 Phase,404751250008002,fd00:0000:0800:DBBA:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,nan,nan,nan,nan,nan,GP5015712,fd00:0000:0800:DBBA:0000:0000:0000:0001,Refurbished,SBPDCL


In [ ]:
# Convert all columns to string data type

# change into str
warehouse_meter_imsi_data = warehouse_meter_imsi_data.astype(str)

# ----------------------------------------------------------------------------------

# change into upper
warehouse_meter_imsi_data.columns = warehouse_meter_imsi_data.columns.str.upper()

# ----------------------------------------------------------------------------------


# Remove leading and trailing whitespaces from all columns where dtype is 'O'
warehouse_meter_imsi_data = warehouse_meter_imsi_data.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)



# SIM SET CHECK STATUS

In [ ]:
def fun_sim_status_check(row):
    if (row['IMSI'] == row['METER_MATCH_UDC_ID']) and (row['WAREHOUSE_UTILITY'] == row['METER_MATCH_UTILITY']):
        return "SAME SET IN SAME UTILITY"
    elif (row['IMSI'] != row['METER_MATCH_UDC_ID']) and (row['WAREHOUSE_UTILITY'] == row['METER_MATCH_UTILITY']):
        return "DIFFERENT SET IN SAME UTILITY"
    elif (row['IMSI'] == row['METER_MATCH_UDC_ID']) and (row['WAREHOUSE_UTILITY'] != row['METER_MATCH_UTILITY']):
        return "SAME SET IN DIFFERENT UTILITY"
    elif (row['IMSI'] != row['METER_MATCH_UDC_ID']) and (row['WAREHOUSE_UTILITY'] != row['METER_MATCH_UTILITY']):
        return "DIFFERENT SET IN DIFFERENT UTILITY"
    else:
        return None

# Apply the function to create a new column
warehouse_meter_imsi_data['SIM SET CHECK STATUS'] = warehouse_meter_imsi_data.apply(fun_sim_status_check, axis=1)



# TRUE - FALSE COLUMN

In [ ]:
def fun_check_true_false__METER_SIM(row):
    if row['IMSI']==row['METER_MATCH_UDC_ID']:
        return True
    else:
        return False
    
warehouse_meter_imsi_data["METER_SIM"] =  warehouse_meter_imsi_data.apply(fun_check_true_false__METER_SIM, axis=1)

In [ ]:
def fun_check_true_false_METER_ORG(row):
    if row['METER_MATCH_UTILITY']==row['WAREHOUSE_UTILITY']:
        return True
    else:
        return False
    
warehouse_meter_imsi_data["METER_ORG"] =  warehouse_meter_imsi_data.apply(fun_check_true_false_METER_ORG, axis=1)

In [ ]:
def fun_check_true_false_SIM_ORG(row):
    if row['IMSI_MATCHED_UTILITY']==row['WAREHOUSE_UTILITY']:
        return True
    else:
        return False
    
warehouse_meter_imsi_data["SIM_ORG"] =  warehouse_meter_imsi_data.apply(fun_check_true_false_SIM_ORG, axis=1)

In [ ]:
def fun_check_true_false_METER_IP(row):
    if row['METER_MATCH_IP']==row['IP']:
        return True
    else:
        return False
    
warehouse_meter_imsi_data["METER_IP"] =  warehouse_meter_imsi_data.apply(fun_check_true_false_METER_IP, axis=1)

# SIM SET CHECK STATUS RECHECK COLUMN

In [ ]:
def check_set_and_utility(row):
    if row['METER_SIM'] == True and ((row['METER_ORG'] == True) and (row["SIM_ORG"] == True)) :
        return "SAME SET IN SAME UTILITY"
    elif row['METER_SIM'] == True and ((row['METER_ORG'] == False) and (row["SIM_ORG"] == False)) :
        return "SAME SET IN DIFFERENT UTILITY"
    elif row['METER_SIM'] == False and ((row['METER_ORG'] == True) and (row["SIM_ORG"] == True)) :
        return "DIFFERENT SET IN SAME UTILITY"
    elif row['METER_SIM'] == False and ((row['METER_ORG'] == False) and (row["SIM_ORG"] == False)) :
        return "DIFFERENT SET IN DIFFERENT UTILITY"
    elif row['METER_SIM']== False and row['METER_ORG']== True and row['SIM_ORG']== False and row['METER_IP']== False:
        return "REFURBISHED METER WITH NEW IMSI"
    else:
        return None


warehouse_meter_imsi_data["SIM SET CHECK STATUS RECHECK"] =  warehouse_meter_imsi_data.apply(check_set_and_utility, axis=1)


In [ ]:
warehouse_meter_imsi_data.columns

Index(['METER NO.', 'BOX NUMBER', 'FINAL SIM', 'IME', 'PHASE', 'IMSI', 'IP',
       'FINAL HES MAPPING', 'MAKE', 'LOT BY LOT', 'IMPORTATION STATUS',
       'MFG_SERIAL_NUM', 'METER_MATCH_UDC_ID', 'METER_MATCH_IP',
       'METER_MATCH_STATUS', 'METER_MATCH_UTILITY', 'IMSI_MATCHED_METER_NO',
       'IMSI_MATCHED_IP', 'IMSI_MATCHED_STATUS', 'IMSI_MATCHED_UTILITY',
       'SIM SET CHECK STATUS', 'METER_SIM', 'METER_ORG', 'SIM_ORG', 'METER_IP',
       'SIM SET CHECK STATUS RECHECK'],
      dtype='object')

# Action taken column

In [ ]:

def fun_action_taken_case_I(row):

    # case 1
    
    if (row['METER_MATCH_STATUS'] == "Installed")  & (row['IMSI_MATCHED_STATUS'] == "Installed"):
        return "SET IS IN INSTALLED STAGE"
    elif (row['METER_MATCH_STATUS'] != "Installed")  & (row['IMSI_MATCHED_STATUS'] == "Installed"):
        return "IMSI IS IN INSTALLED STAGE"
    elif (row['METER_MATCH_STATUS'] == "Installed")  & (row['IMSI_MATCHED_STATUS'] != "Installed"):
        return "METER IS IN INSTALLED STAGE"
    
    # case 2 


    # set 1 and row['METER_IP'] == True
    elif (row['METER_SIM'] == True) and ((row['METER_ORG']  == True) and (row["SIM_ORG"] == True)) and (row['METER_IP'] == True):
        return "METER-SIM LINK FILE REQUIRED"
    
    # set 1 and row['METER_IP'] == False 
    elif (row['METER_SIM'] == True) and ((row['METER_ORG'] == True) and (row["SIM_ORG"] == True)) and (row['METER_IP'] == False) :
        return "SIM AND METER LINK FILE REQUIRED"
    
    # set 2  "SAME SET IN DIFFERENT UTILITY"
    elif (row['METER_SIM'] == True) and ((row['METER_ORG'] == False) and (row["SIM_ORG"] == False)) :
        return "METER , SIM AND METER-SIM LINK FILE REQUIRED"

    #set 3  "DIFFERENT SET IN SAME UTILITY"
    
    elif (row['METER_SIM'] == False) and ((row['METER_ORG'] == True) and (row["SIM_ORG"] == True)) :
        return "SIM AND METER-SIM LINK FILE REQUIRED"
    
    # set 4  "DIFFERENT SET IN DIFFERENT UTILITY"
    elif (row['METER_SIM'] == False) and ((row['METER_ORG'] == False) and (row["SIM_ORG"] == False)) :
        return "METER , SIM AND METER-SIM LINK FILE REQUIRED"
    


    # new cases are added in this to handle imsi null
    
    elif ( row['IMSI_MATCHED_METER_NO'] =='nan') and ( row['IMSI_MATCHED_IP'] =='nan') and ( row['IMSI_MATCHED_STATUS'] =='nan') and (row['METER_ORG'] == True ):
        return "SIM AND METER-SIM LINK FILE REQUIRED"
    elif ( row['IMSI_MATCHED_METER_NO'] =='nan' ) and ( row['IMSI_MATCHED_IP'] =='nan') and ( row['IMSI_MATCHED_STATUS'] =='nan') and (row['METER_ORG'] == False ):
        return "METER , SIM AND METER-SIM LINK FILE REQUIRED"
    elif ( row['IMSI_MATCHED_METER_NO'] =='nan') and ( row['IMSI_MATCHED_IP'] =='nan') and ( row['IMSI_MATCHED_STATUS'] =='nan')  and ( row['METER_MATCH_UDC_ID'] =='nan') and ( row['METER_MATCH_IP'] =='nan') and ( row['METER_MATCH_STATUS'] =='nan'):
        return "METER , SIM AND METER-SIM LINK FILE REQUIRED"
    
    

    
    else:
        return " NEED TO CHECK THIS FILE"
    
    
    
warehouse_meter_imsi_data["ACTION TAKEN"] =  warehouse_meter_imsi_data.apply(fun_action_taken_case_I, axis=1)

In [ ]:
warehouse_meter_imsi_data[warehouse_meter_imsi_data['IMSI_MATCHED_METER_NO']!='nan'].count()

,METER NO.,BOX NUMBER,FINAL SIM,IME,PHASE,IMSI,IP,FINAL HES MAPPING,MAKE,LOT BY LOT,IMPORTATION STATUS,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
0,GP4971073,BOX NO- 1805P,8991759065212008001,869912032848513,1 Phase,404751250008001,fd00:0000:0800:DBB9:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,GP4971073,404751250453398,2001:4490:0800:ACA4:0000:0000:0000:0001,Retired,NBPDCL,GP4971073,fd00:0000:0800:DBB9:0000:0000:0000:0001,Meter Shop,SBPDCL,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,NEED TO CHECK THIS FILE
1,GP5015712,BOX NO- 1805P,8991759065212008002,869912033630167,1 Phase,404751250008002,fd00:0000:0800:DBBA:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,nan,nan,nan,nan,nan,GP5015712,fd00:0000:0800:DBBA:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"
2,GP5015622,BOX NO- 1805P,8991759065212008003,869912033743713,1 Phase,404751250008003,fd00:0000:0800:DBBB:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,nan,nan,nan,nan,nan,GP5015622,fd00:0000:0800:DBBB:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"
3,GP5019777,BOX NO- 1805P,8991759065212008004,869912033458346,1 Phase,404751250008004,fd00:0000:0800:DBBC:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,nan,nan,nan,nan,nan,GP5019777,fd00:0000:0800:DBBC:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"
4,GP5011165,BOX NO- 1805P,8991759065212008005,869912033521200,1 Phase,404751250008005,fd00:0000:0800:DBBD:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,nan,nan,nan,nan,nan,GP5011165,fd00:0000:0800:DBBD:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,GP5047914,BOX NO-L490,8991759065212011962,869912036770598,1 Phase,404751250011962,fd00:0000:0800:EB32:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,HOLD,nan,nan,nan,nan,nan,GP5047914,fd00:0000:0800:EB32:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"
3996,GP4962506,BOX NO-L490,8991759065212011963,869912031765965,1 Phase,404751250011963,fd00:0000:0800:EB33:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,HOLD,nan,nan,nan,nan,nan,GP4962506,fd00:0000:0800:EB33:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"
3997,GP5029262,BOX NO-L490,8991759065212011964,869912032828671,1 Phase,404751250011964,fd00:0000:0800:EB34:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,HOLD,nan,nan,nan,nan,nan,GP5029262,fd00:0000:0800:EB34:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"
3998,GP4964817,BOX NO-L490,8991759065212011965,869912039773490,1 Phase,404751250011965,fd00:0000:0800:EB35:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,HOLD,nan,nan,nan,nan,nan,GP4964817,fd00:0000:0800:EB35:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"


In [ ]:
warehouse_meter_imsi_data.head(2)

,METER NO.,BOX NUMBER,FINAL SIM,IME,PHASE,IMSI,IP,FINAL HES MAPPING,MAKE,LOT BY LOT,IMPORTATION STATUS,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
0,GP4971073,BOX NO- 1805P,8991759065212008001,869912032848513,1 Phase,404751250008001,fd00:0000:0800:DBB9:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,GP4971073,404751250453398,2001:4490:0800:ACA4:0000:0000:0000:0001,Retired,NBPDCL,GP4971073,fd00:0000:0800:DBB9:0000:0000:0000:0001,Meter Shop,SBPDCL,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,NEED TO CHECK THIS FILE
1,GP5015712,BOX NO- 1805P,8991759065212008002,869912033630167,1 Phase,404751250008002,fd00:0000:0800:DBBA:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,nan,nan,nan,nan,nan,GP5015712,fd00:0000:0800:DBBA:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"


In [ ]:
"""

def fun_action_taken_new_case(row):

    if (row['IMSI_MATCHED_METER_NO'].isnull()) & (row['IMSI_MATCHED_IP'].isnull()) & (row['IMSI_MATCHED_STATUS'].isnull()) and (row['METER_ORG'] == True ):
        return "SIM AND METER SIM LINK REQUIRED"
    elif (row['IMSI_MATCHED_METER_NO'].isnull()) & (row['IMSI_MATCHED_IP'].isnull()) & (row['IMSI_MATCHED_STATUS'].isnull()) and (row['METER_ORG'] == False) :
        return "METER , SIM AND METER SIM LINK REQUIRED"
        
    elif (row['IMSI_MATCHED_METER_NO'].isnull()) & (row['IMSI_MATCHED_IP'].isnull()) & (row['IMSI_MATCHED_STATUS'].isnull()) and (row['METER_ORG'] == True ) and (row['METER_MATCH_UDC_ID'].isnull()) &  (row['METER_MATCH_IP'].isnull()) &  (row['METER_MATCH_STATUS'].isnull()):
        return "METER , SIM AND METER SIM LINK REQUIRED"
    else:
        return "IMSI IS NULL DID NOT MATCH AS PER CASES WRITTEN"


warehouse_meter_imsi_data["ACTION TAKEN"] =  warehouse_meter_imsi_data.apply(fun_action_taken_new_case, axis=1)

"""


'\n\ndef fun_action_taken_new_case(row):\n\n    if (row[\'IMSI_MATCHED_METER_NO\'].isnull()) & (row[\'IMSI_MATCHED_IP\'].isnull()) & (row[\'IMSI_MATCHED_STATUS\'].isnull()) and (row[\'METER_ORG\'] == True ):\n        return "SIM AND METER SIM LINK REQUIRED"\n    elif (row[\'IMSI_MATCHED_METER_NO\'].isnull()) & (row[\'IMSI_MATCHED_IP\'].isnull()) & (row[\'IMSI_MATCHED_STATUS\'].isnull()) and (row[\'METER_ORG\'] == False) :\n        return "METER , SIM AND METER SIM LINK REQUIRED"\n        \n    elif (row[\'IMSI_MATCHED_METER_NO\'].isnull()) & (row[\'IMSI_MATCHED_IP\'].isnull()) & (row[\'IMSI_MATCHED_STATUS\'].isnull()) and (row[\'METER_ORG\'] == True ) and (row[\'METER_MATCH_UDC_ID\'].isnull()) &  (row[\'METER_MATCH_IP\'].isnull()) &  (row[\'METER_MATCH_STATUS\'].isnull()):\n        return "METER , SIM AND METER SIM LINK REQUIRED"\n    else:\n        return "IMSI IS NULL DID NOT MATCH AS PER CASES WRITTEN"\n\n\nwarehouse_meter_imsi_data["ACTION TAKEN"] =  warehouse_meter_imsi_data.apply

In [ ]:
"""

def fun_action_taken_new_case(row):
    if (not row['IMSI_MATCHED_METER_NO']) and (not row['IMSI_MATCHED_IP']) and (not row['IMSI_MATCHED_STATUS']) and (row['METER_ORG'] == True ):
        return "SIM AND METER SIM LINK REQUIRED"
    elif (not row['IMSI_MATCHED_METER_NO']) and (not row['IMSI_MATCHED_IP']) and (not row['IMSI_MATCHED_STATUS']) and (row['METER_ORG'] == False ):
        return "METER , SIM AND METER SIM LINK REQUIRED"
    elif (not row['IMSI_MATCHED_METER_NO']) and (not row['IMSI_MATCHED_IP']) and (not row['IMSI_MATCHED_STATUS'])  and (not row['METER_MATCH_UDC_ID']) and (not row['METER_MATCH_IP']) and (not row['METER_MATCH_STATUS']):
        return "METER , SIM AND METER SIM LINK REQUIRED"
    else:
        return "IMSI IS NULL DID NOT MATCH AS PER CASES WRITTEN"

warehouse_meter_imsi_data["ACTION TAKEN"] = warehouse_meter_imsi_data.apply(fun_action_taken_new_case, axis=1)


"""


'\n\ndef fun_action_taken_new_case(row):\n    if (not row[\'IMSI_MATCHED_METER_NO\']) and (not row[\'IMSI_MATCHED_IP\']) and (not row[\'IMSI_MATCHED_STATUS\']) and (row[\'METER_ORG\'] == True ):\n        return "SIM AND METER SIM LINK REQUIRED"\n    elif (not row[\'IMSI_MATCHED_METER_NO\']) and (not row[\'IMSI_MATCHED_IP\']) and (not row[\'IMSI_MATCHED_STATUS\']) and (row[\'METER_ORG\'] == False ):\n        return "METER , SIM AND METER SIM LINK REQUIRED"\n    elif (not row[\'IMSI_MATCHED_METER_NO\']) and (not row[\'IMSI_MATCHED_IP\']) and (not row[\'IMSI_MATCHED_STATUS\'])  and (not row[\'METER_MATCH_UDC_ID\']) and (not row[\'METER_MATCH_IP\']) and (not row[\'METER_MATCH_STATUS\']):\n        return "METER , SIM AND METER SIM LINK REQUIRED"\n    else:\n        return "IMSI IS NULL DID NOT MATCH AS PER CASES WRITTEN"\n\nwarehouse_meter_imsi_data["ACTION TAKEN"] = warehouse_meter_imsi_data.apply(fun_action_taken_new_case, axis=1)\n\n\n'

# Finally done

In [ ]:
warehouse_meter_imsi_data.head(1)

,METER NO.,BOX NUMBER,FINAL SIM,IME,PHASE,IMSI,IP,FINAL HES MAPPING,MAKE,LOT BY LOT,IMPORTATION STATUS,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
0,GP4971073,BOX NO- 1805P,8991759065212008001,869912032848513,1 Phase,404751250008001,fd00:0000:0800:DBB9:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,GP4971073,404751250453398,2001:4490:0800:ACA4:0000:0000:0000:0001,Retired,NBPDCL,GP4971073,fd00:0000:0800:DBB9:0000:0000:0000:0001,Meter Shop,SBPDCL,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,NEED TO CHECK THIS FILE


# FINAL FILE SAVE AT MULTIPLE LOCATION

In [ ]:
# Final file at desktop folder

warehouse_meter_imsi_data.to_csv(r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\warehouse_Data_meter_sim_final.csv',index=False)


# Final file in code folder

warehouse_meter_imsi_data.to_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\warehouse_Data_meter_sim_final.csv",index=False)

In [ ]:
warehouse_meter_imsi_data_check=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\warehouse_Data_meter_sim_final.csv")

In [ ]:
# Convert all columns to string data type

# change into str
warehouse_meter_imsi_data_check = warehouse_meter_imsi_data_check.astype(str)

# ----------------------------------------------------------------------------------

# change into upper
warehouse_meter_imsi_data_check.columns = warehouse_meter_imsi_data_check.columns.str.upper()

# ----------------------------------------------------------------------------------

# Remove leading and trailing whitespaces from all columns where dtype is 'O'
warehouse_meter_imsi_data_check = warehouse_meter_imsi_data_check.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)

# NEED TO CHECK FILE

In [ ]:
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM = warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['ACTION TAKEN']==' NEED TO CHECK THIS FILE']

In [ ]:
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM.to_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\NEED_TO_CHECK_THIS_FILE_WAREHOUSE_METER_SIM.csv",index=False)
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM.to_csv(r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_FOLDER\OUTPUT_DATA_FILE\NEED_TO_CHECK_THIS_FILE_WAREHOUSE_METER_SIM.csv',index=False)



In [ ]:
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM_CSV = pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\NEED_TO_CHECK_THIS_FILE_WAREHOUSE_METER_SIM.csv")

In [ ]:
NEED_TO_CHECK_FILE_WAREHOUSE_METER_SIM_CSV.head()

,METER NO.,BOX NUMBER,FINAL SIM,IME,PHASE,IMSI,IP,FINAL HES MAPPING,MAKE,LOT BY LOT,IMPORTATION STATUS,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN


In [ ]:
warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['ACTION TAKEN'].isnull()]

,METER NO.,BOX NUMBER,FINAL SIM,IME,PHASE,IMSI,IP,FINAL HES MAPPING,MAKE,LOT BY LOT,IMPORTATION STATUS,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN


In [ ]:
warehouse_meter_imsi_data_check.head()


,METER NO.,BOX NUMBER,FINAL SIM,IME,PHASE,IMSI,IP,FINAL HES MAPPING,MAKE,LOT BY LOT,IMPORTATION STATUS,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
0,GP4971073,BOX NO- 1805P,8991759065212008001,869912032848513,1 Phase,404751250008001,fd00:0000:0800:DBB9:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,GP4971073,404751250453398.0,2001:4490:0800:ACA4:0000:0000:0000:0001,Retired,NBPDCL,GP4971073,fd00:0000:0800:DBB9:0000:0000:0000:0001,Meter Shop,SBPDCL,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,NEED TO CHECK THIS FILE
1,GP5015712,BOX NO- 1805P,8991759065212008002,869912033630167,1 Phase,404751250008002,fd00:0000:0800:DBBA:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,nan,nan,nan,nan,nan,GP5015712,fd00:0000:0800:DBBA:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"
2,GP5015622,BOX NO- 1805P,8991759065212008003,869912033743713,1 Phase,404751250008003,fd00:0000:0800:DBBB:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,nan,nan,nan,nan,nan,GP5015622,fd00:0000:0800:DBBB:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"
3,GP5019777,BOX NO- 1805P,8991759065212008004,869912033458346,1 Phase,404751250008004,fd00:0000:0800:DBBC:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,nan,nan,nan,nan,nan,GP5019777,fd00:0000:0800:DBBC:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"
4,GP5011165,BOX NO- 1805P,8991759065212008005,869912033521200,1 Phase,404751250008005,fd00:0000:0800:DBBD:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,nan,nan,nan,nan,nan,GP5011165,fd00:0000:0800:DBBD:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"


In [ ]:
warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['ACTION TAKEN'] != "NEED TO CHECK THIS FILE"]

,METER NO.,BOX NUMBER,FINAL SIM,IME,PHASE,IMSI,IP,FINAL HES MAPPING,MAKE,LOT BY LOT,IMPORTATION STATUS,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
1,GP5015712,BOX NO- 1805P,8991759065212008002,869912033630167,1 Phase,404751250008002,fd00:0000:0800:DBBA:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,nan,nan,nan,nan,nan,GP5015712,fd00:0000:0800:DBBA:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"
2,GP5015622,BOX NO- 1805P,8991759065212008003,869912033743713,1 Phase,404751250008003,fd00:0000:0800:DBBB:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,nan,nan,nan,nan,nan,GP5015622,fd00:0000:0800:DBBB:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"
3,GP5019777,BOX NO- 1805P,8991759065212008004,869912033458346,1 Phase,404751250008004,fd00:0000:0800:DBBC:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,nan,nan,nan,nan,nan,GP5019777,fd00:0000:0800:DBBC:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"
4,GP5011165,BOX NO- 1805P,8991759065212008005,869912033521200,1 Phase,404751250008005,fd00:0000:0800:DBBD:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,nan,nan,nan,nan,nan,GP5011165,fd00:0000:0800:DBBD:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"
5,GP5020927,BOX NO- 1805P,8991759065212008006,860650040408685,1 Phase,404751250008006,fd00:0000:0800:DBBE:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,nan,nan,nan,nan,nan,GP5020927,fd00:0000:0800:DBBE:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,GP5047914,BOX NO-L490,8991759065212011962,869912036770598,1 Phase,404751250011962,fd00:0000:0800:EB32:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,HOLD,nan,nan,nan,nan,nan,GP5047914,fd00:0000:0800:EB32:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"
3996,GP4962506,BOX NO-L490,8991759065212011963,869912031765965,1 Phase,404751250011963,fd00:0000:0800:EB33:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,HOLD,nan,nan,nan,nan,nan,GP4962506,fd00:0000:0800:EB33:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"
3997,GP5029262,BOX NO-L490,8991759065212011964,869912032828671,1 Phase,404751250011964,fd00:0000:0800:EB34:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,HOLD,nan,nan,nan,nan,nan,GP5029262,fd00:0000:0800:EB34:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"
3998,GP4964817,BOX NO-L490,8991759065212011965,869912039773490,1 Phase,404751250011965,fd00:0000:0800:EB35:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,HOLD,nan,nan,nan,nan,nan,GP4964817,fd00:0000:0800:EB35:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"


In [ ]:
warehouse_meter_imsi_data_check['ACTION TAKEN'].unique()

array(['NEED TO CHECK THIS FILE',
       'METER,SIM AND METER-SIM LINK FILE REQUIRED',
       'METER IS IN INSTALLED STAGE'], dtype=object)

In [ ]:
warehouse_meter_imsi_data_check.head(2)

,METER NO.,BOX NUMBER,FINAL SIM,IME,PHASE,IMSI,IP,FINAL HES MAPPING,MAKE,LOT BY LOT,IMPORTATION STATUS,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
0,GP4971073,BOX NO- 1805P,8991759065212008001,869912032848513,1 Phase,404751250008001,fd00:0000:0800:DBB9:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,GP4971073,404751250453398.0,2001:4490:0800:ACA4:0000:0000:0000:0001,Retired,NBPDCL,GP4971073,fd00:0000:0800:DBB9:0000:0000:0000:0001,Meter Shop,SBPDCL,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,NEED TO CHECK THIS FILE
1,GP5015712,BOX NO- 1805P,8991759065212008002,869912033630167,1 Phase,404751250008002,fd00:0000:0800:DBBA:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,nan,nan,nan,nan,nan,GP5015712,fd00:0000:0800:DBBA:0000:0000:0000:0001,Refurbished,SBPDCL,DIFFERENT SET IN DIFFERENT UTILITY,False,False,False,False,DIFFERENT SET IN DIFFERENT UTILITY,"METER,SIM AND METER-SIM LINK FILE REQUIRED"


In [ ]:
warehouse_meter_imsi_data_check['ACTION TAKEN'].unique()

array(['NEED TO CHECK THIS FILE',
       'METER,SIM AND METER-SIM LINK FILE REQUIRED',
       'METER IS IN INSTALLED STAGE'], dtype=object)

In [ ]:
warehouse_meter_imsi_data_check['SIM SET CHECK STATUS RECHECK'].unique()

array(['REFURBISHED METER WITH NEW IMSI',
       'DIFFERENT SET IN DIFFERENT UTILITY'], dtype=object)

In [ ]:
warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['IMSI_MATCHED_METER_NO']=="nan"].count()

METER NO.                       4
BOX NUMBER                      4
FINAL SIM                       4
IME                             4
PHASE                           4
IMSI                            4
IP                              4
FINAL HES MAPPING               4
MAKE                            4
LOT BY LOT                      4
IMPORTATION STATUS              4
MFG_SERIAL_NUM                  4
METER_MATCH_UDC_ID              4
METER_MATCH_IP                  4
METER_MATCH_STATUS              4
METER_MATCH_UTILITY             4
IMSI_MATCHED_METER_NO           4
IMSI_MATCHED_IP                 4
IMSI_MATCHED_STATUS             4
IMSI_MATCHED_UTILITY            4
SIM SET CHECK STATUS            4
METER_SIM                       4
METER_ORG                       4
SIM_ORG                         4
METER_IP                        4
SIM SET CHECK STATUS RECHECK    4
ACTION TAKEN                    4
dtype: int64

In [ ]:
warehouse_meter_imsi_data_check['METER_MATCH_UDC_ID'].dtype

dtype('O')

In [ ]:
warehouse_meter_imsi_data_check['ACTION TAKEN'].unique()

array(['NEED TO CHECK THIS FILE',
       'METER,SIM AND METER-SIM LINK FILE REQUIRED',
       'METER IS IN INSTALLED STAGE'], dtype=object)

In [ ]:
warehouse_meter_imsi_data_check[warehouse_meter_imsi_data_check['ACTION TAKEN']=='NEED TO CHECK THIS FILE'].count()

METER NO.                       1187
BOX NUMBER                      1187
FINAL SIM                       1187
IME                             1187
PHASE                           1187
IMSI                            1187
IP                              1187
FINAL HES MAPPING               1187
MAKE                            1187
LOT BY LOT                      1187
IMPORTATION STATUS              1187
MFG_SERIAL_NUM                  1187
METER_MATCH_UDC_ID              1187
METER_MATCH_IP                  1187
METER_MATCH_STATUS              1187
METER_MATCH_UTILITY             1187
IMSI_MATCHED_METER_NO           1187
IMSI_MATCHED_IP                 1187
IMSI_MATCHED_STATUS             1187
IMSI_MATCHED_UTILITY            1187
SIM SET CHECK STATUS            1187
METER_SIM                       1187
METER_ORG                       1187
SIM_ORG                         1187
METER_IP                        1187
SIM SET CHECK STATUS RECHECK    1187
ACTION TAKEN                    1187
d

In [ ]:
#assest_initiation_invent_sb[assest_initiation_invent_sb['UDC_ID']=='404751250008001']

,METERNUMBER,MFG_SERIAL_NUM,UDC_ID,NETWORK_ID,INSTALLATIONNUMBER,STATUS,INSERT_TIME,"DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"
219412,GOEGP4971073,GP4971073,404751250008001,fd00:0000:0800:DBB9:0000:0000:0000:0001,DALGND-49,Meter Shop,12/17/2023 5:00:50 PM,SBPDCL
932891,GOEGP4971073,GP4971073,404751250008001,fd00:0000:0800:DBB9:0000:0000:0000:0001,nan,Meter Shop,12/17/2023 5:00:50 PM,SBPDCL


In [ ]:
#assest_initiation_invent_nb[assest_initiation_invent_nb['UDC_ID']=='404751250008001']

,METERNUMBER,MFG_SERIAL_NUM,UDC_ID,NETWORK_ID,INSTALLATIONNUMBER,STATUS,INSERT_TIME,"DECODE(D.ORG_ID,52,'SBPDCL',102,'NBPDCL')"


In [ ]:
df ad da asd

# creating meter-sim link file

In [ ]:
# Convert all columns to string data type

# change into str
warehouse_meter_imsi_data_check = warehouse_meter_imsi_data_check.astype(str)
# ----------------------------------------------------------------------------------
# change into upper
warehouse_meter_imsi_data_check.columns = warehouse_meter_imsi_data_check.columns.str.upper()
# ----------------------------------------------------------------------------------
# Remove leading and trailing whitespaces from all columns where dtype is 'O'
warehouse_meter_imsi_data_check = warehouse_meter_imsi_data_check.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)



In [ ]:
# creating meter_sim_link_file

create_meter_sim_link_file= warehouse_meter_imsi_data_check[(warehouse_meter_imsi_data_check['ACTION TAKEN']=='METER,SIM AND METER-SIM LINK FILE REQUIRED') | (warehouse_meter_imsi_data_check['ACTION TAKEN']=='METER IS IN INSTALLED STAGE') | (warehouse_meter_imsi_data_check['ACTION TAKEN']=='NEED TO CHECK THIS FILE')]

In [ ]:
create_meter_sim_link_file['ACTION TAKEN'].unique()


array(['NEED TO CHECK THIS FILE',
       'METER,SIM AND METER-SIM LINK FILE REQUIRED',
       'METER IS IN INSTALLED STAGE'], dtype=object)

In [ ]:
print("count")

create_meter_sim_link_file.count()[0]

count


C:\Users\Ratan Kumar Jha\AppData\Local\Temp\ipykernel_22944\540428121.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  create_meter_sim_link_file.count()[0]


4000

In [ ]:
create_meter_sim_link_file.head(1)

,METER NO.,BOX NUMBER,FINAL SIM,IME,PHASE,IMSI,IP,FINAL HES MAPPING,MAKE,LOT BY LOT,IMPORTATION STATUS,MFG_SERIAL_NUM,METER_MATCH_UDC_ID,METER_MATCH_IP,METER_MATCH_STATUS,METER_MATCH_UTILITY,IMSI_MATCHED_METER_NO,IMSI_MATCHED_IP,IMSI_MATCHED_STATUS,IMSI_MATCHED_UTILITY,SIM SET CHECK STATUS,METER_SIM,METER_ORG,SIM_ORG,METER_IP,SIM SET CHECK STATUS RECHECK,ACTION TAKEN
0,GP4971073,BOX NO- 1805P,8991759065212008001,869912032848513,1 Phase,404751250008001,fd00:0000:0800:DBB9:0000:0000:0000:0001,NBPDCL,Genus 3G,LOT 2,Imported,GP4971073,404751250453398.0,2001:4490:0800:ACA4:0000:0000:0000:0001,Retired,NBPDCL,GP4971073,fd00:0000:0800:DBB9:0000:0000:0000:0001,Meter Shop,SBPDCL,DIFFERENT SET IN SAME UTILITY,False,True,False,False,REFURBISHED METER WITH NEW IMSI,NEED TO CHECK THIS FILE


In [ ]:

# Custom function to update meter numbers
def update_meter_number(meter_number):
    if meter_number.startswith('GP') or meter_number.startswith('GL'):
        return 'GOE' + meter_number
    elif meter_number.startswith('LT'):
        return 'LNT' + meter_number
    else:
        return meter_number

# Apply the custom function to each element in the 'METER NO.' column
create_meter_sim_link_file['NEW_METER_NUMBER'] = create_meter_sim_link_file['METER NO.'].apply(update_meter_number)




In [ ]:
def fun_meter_sim_link_file(row):

    return "{};EP2PMeter;{};{};{}".format(row['NEW_METER_NUMBER'], row['IMSI'],row['FINAL SIM'],row['FINAL HES MAPPING'])

create_meter_sim_link_file['meter_sim_link_file_single_file_data'] = create_meter_sim_link_file.apply(fun_meter_sim_link_file, axis=1)

In [ ]:
create_meter_sim_link_file.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\new\create_meter_sim_link_file.csv",index=False)

# CREATING A NEW DATA FRAME

In [ ]:
single_row_data = pd.DataFrame()
single_row_data

""


In [ ]:
single_row_data=create_meter_sim_link_file['meter_sim_link_file_single_file_data']

In [ ]:
single_row_data.to_csv("EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv",header=None,index=False)

In [ ]:
EESL_GENUS_PO0000000403_00000_SIM_Meter_Link=pd.read_csv(r"C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv",header=None)

In [ ]:
EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.head()

,0
0,GOEGP4971073;EP2PMeter;404751250008001;8991759...
1,GOEGP5015712;EP2PMeter;404751250008002;8991759...
2,GOEGP5015622;EP2PMeter;404751250008003;8991759...
3,GOEGP5019777;EP2PMeter;404751250008004;8991759...
4,GOEGP5011165;EP2PMeter;404751250008005;8991759...


In [ ]:
new_df_check = EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.loc[2]
#print(f"len of new_df_check: {new_df_check}")
a=';'
word=""
for i in new_df_check:
    print(i)
    for j in i:
        word=word+j
        if a in j:
            #print(a)
            #print(f"word is : {word}")
            word=""
            #print("semicolon is here")
print(f"last word is : {word}")


GOEGP5015622;EP2PMeter;404751250008003;8991759065212008003;NBPDCL
last word is : NBPDCL


In [ ]:
input_string = "GOEGP5015622;EP2PMeter;404751250008003;8991759065212008003;NBPDCL"

# Split the input string by semicolon and get the last element
last_word = input_string.split(';')[-1]



Last word is: NBPDCL


# create multiple single file

In [ ]:
import pandas as pd

import os,sys
from datetime import datetime

k=100
# Load your file into a pandas DataFrame
data_frame_concat_data = pd.read_csv(r'C:\PYTHON\EDF_CODE_AUTOMATION_PYTHON\METER_MATCHING_DATA\EESL_GENUS_PO0000000403_00000_SIM_Meter_Link.csv', header=None)  # Replace 'your_input_file.csv' with your actual file name

base_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\data_insert_single_file'

num_folder = 0
new_folder_name = "EESL_GENUS_SINGLE_METER"

new_folder_path = os.path.join(base_path, new_folder_name)
os.makedirs(new_folder_path)

def get_current_time_stamp():
    return f"{datetime.now().strftime('%Y%m%d_%H%M%S')}"

CURRENT_TIME_STAMP = get_current_time_stamp()

# Iterate through each row and save it as a separate file
for index, row in data_frame_concat_data.iterrows():
    
    # Create a new DataFrame with the current row
    single_row_df = pd.DataFrame([row])
    print(f"single_row_data : {single_row_df}")

    CURRENT_TIME_STAMP = get_current_time_stamp()
    #print(CURRENT_TIME_STAMP)
    
    print(k)
    print("-----------------")
    
    # Create the full path for the new folder

    if k > 995:
        k = 100
        num_folder = num_folder + 1
        new_folder_name = "EESL_GENUS_SINGLE_METER_" + str(num_folder)
        new_folder_path = os.path.join(base_path, new_folder_name)
        os.makedirs(new_folder_path)
    else:
        file_name = f'EESL_GENUS_PO0000000{k}_00000_SIM_Meter_Link_{CURRENT_TIME_STAMP}_NBPDCL.csv'
        file_path = os.path.join(new_folder_path, file_name)
        #single_row_df.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\satyam_data\single_file_data\file_name", index=False, header=False) 
        single_row_df.to_csv(file_path, index=False, header=False)
        k=k+1

    break
print("METER-SIM LINK FILE DONE")

20231221_123812
100
-----------------
20231221_123812
101
-----------------
20231221_123812
102
-----------------
20231221_123812
103
-----------------
20231221_123812
104
-----------------
20231221_123813
105
-----------------
20231221_123813
106
-----------------
20231221_123813
107
-----------------
20231221_123813
108
-----------------
20231221_123813
109
-----------------
20231221_123813
110
-----------------
20231221_123813
111
-----------------
20231221_123813
112
-----------------
20231221_123813
113
-----------------
20231221_123813
114
-----------------
20231221_123813
115
-----------------
20231221_123813
116
-----------------
20231221_123813
117
-----------------
20231221_123813
118
-----------------
20231221_123813
119
-----------------
20231221_123813
120
-----------------
20231221_123813
121
-----------------
20231221_123813
122
-----------------
20231221_123813
123
-----------------
20231221_123813
124
-----------------
20231221_123813
125
-----------------
20231221_123

'002;8991759...\nName: 1, dtype: object'

'12;EP'

# change into zip file
###### C:\Users\Ratan Kumar Jha\Desktop\EESL_GENUS_PO0000000\data_insert_single_file

In [ ]:
import shutil

# Specify the folder path
folder_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\data_insert_single_file'

# Specify the output zip file path
output_zip_path = r'C:\Users\Ratan Kumar Jha\Desktop\METER_IMPORTATION_SINGLE_FILE\data_insert_single_file.zip'

# Create a zip file from the folder
shutil.make_archive(output_zip_path, 'zip', folder_path)

print(f"Zip file created at: {output_zip_path}")


Zip file created at: C:\Users\Ratan Kumar Jha\Desktop\EESL_GENUS_PO0000000\data_insert_single_file.zip


#### METER,SIM AND METER-SIM LINK
    - METER FILE PREPARE
    - SIM FILE PREPARE
    - SIM-METER LINK

#### SIM AND METER-SIM LINK FILE
    - SIM FILE
    - METER-SIM LINK

#### METER-SIM LINK
    - METER-SIM LINK

